# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [5]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# [回答1] 

In [6]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
from IPython.display import display

features = [
    ('int', int_features),
    ('float', float_features),
    ('object', object_features)
]

operations = [
    ('mean', lambda x: x.mean()),
    ('max', lambda x: x.max()),
    ('nunique', lambda x: x.nunique())
]

for dtype, feature in features:
    for operation, func in operations:
        result = func(df[feature])
        print(f"\n--- {dtype} / {operation}---")
        print(result)
        if result.shape[0] == 0:
            print(f"\n{'':=>70}")
            print(f"Something went wrong when running dtype=`{dtype}` with `{operation}`")


--- int / mean---
Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

--- int / max---
Pclass    3
SibSp     8
Parch     9
dtype: int64

--- int / nunique---
Pclass    3
SibSp     7
Parch     8
dtype: int64

--- float / mean---
Age     29.881138
Fare    33.295479
dtype: float64

--- float / max---
Age      80.0000
Fare    512.3292
dtype: float64

--- float / nunique---
Age      98
Fare    281
dtype: int64

--- object / mean---
Series([], dtype: float64)

Something went wrong when running dtype=`object` with `mean`

--- object / max---
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

--- object / nunique---
Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64


我們可以從上表看到，在執行過程中不會有error的出現，只是object型態的資料在運算mean的時候會回傳空值，因為字串並非數字，只能藉由長度或是字母排序取最大值跟相異值，但不能計算平均。

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

1. 之前在課程中有學習到「圖片」的資料，我們面對輸入的資料是圖片時，往往會把它轉成由許多數字所構成的矩陣，因此面對如「圖片」般的比較不同的資料類型，他還是可以變成數值型資料來進行分析。
2. 我認為，時間類特徵中是所有特徵裡面處理起來最複雜的，因為還要考慮到他的週期性、順序等等的因素，並且選擇適當的演算法來使用。